# 独自の MCP サーバーを AgentCore Gateway に統合する

## 概要

数千の開発チームが複数のツールを持つ MCP サーバーを運用している大企業では、ツールの管理と発見が重大な課題となります：

* **ツールの発見と共有**：チームは組織全体でツールを発見し、利用することに苦労しています。各チームは別々の Gateway を維持すべきでしょうか？Gateway URL を共有し、運用オーバーヘッドを生じさせずに中央レジストリを管理するにはどうすればよいでしょうか？
* **Gateway の管理**：各 MCP サーバーに対して別々の Gateway を維持することは、スケールすると管理が困難になります。
* **認証の複雑さ**：複数の MCP サーバー間での認証と認可の管理は、特に機密性の高いエンタープライズデータでは、ますます複雑になります。
* **メンテナンスのオーバーヘッド**：MCP 仕様の更新に追従するには、すべての実装にわたって継続的な作り直しとテストが必要です。

AgentCore Gateway は、Lambda や OpenAPI ツールに加えて、既存の MCP サーバー実装をターゲットとしてオンボードすることをサポートするようになりました。このチュートリアルでは、AgentCore Runtime 上のシンプルな MCP サーバーを使用してこの概念を実演します。

![Diagram](images/mcp-server-target.png)

AgentCore Gateway を MCP サーバーターゲットと統合したら、統合された MCP サーバーを通じてツールを検索し、Strands Agent を使用してツールのリスト取得と呼び出し機能を実演します。これらの各フローの詳細は以下で説明します：

### MCP ターゲットのツール検索動作

AgentCore Gateway の検索機能により、MCP ターゲットを含むすべてのターゲットタイプにわたってツールのセマンティック発見が可能になります。MCP ターゲットの場合、検索機能は同期操作中にキャプチャされインデックス化された正規化されたツール定義に対して動作し、リアルタイムの MCP サーバー通信なしで効率的なセマンティック検索を提供します。ツール定義が MCP ターゲットから同期されると、Gateway は各ツールの名前、説明、パラメータ説明に対して自動的に埋め込みを生成します。これらの埋め込みは正規化されたツール定義と一緒に保存され、検索クエリの意図とコンテキストを理解するセマンティック検索を可能にします。従来のキーワードマッチングとは異なり、正確な用語が一致しなくても関連するツールをエージェントが発見できます。

![Search](images/mcp-server-search-tool.png)

### MCP ターゲットの ListTools 動作

AgentCore Gateway の ListTools 操作は、パフォーマンスと信頼性を優先するキャッシュファーストアプローチに従って、以前に MCP ターゲットから同期されたツール定義へのアクセスを提供します。ツール定義が静的に定義される従来の OpenAPI や Lambda ターゲットとは異なり、MCP ターゲットのツールは同期操作を通じて発見されキャッシュされます。クライアントが ListTools を呼び出すと、Gateway はリアルタイムで MCP サーバーを呼び出すのではなく、永続ストレージからツール定義を取得します。これらの定義は、ターゲット作成/更新時の暗黙的な同期または明示的な SynchronizeGateway API 呼び出しを通じて以前に投入されていました。この操作は正規化されたツール定義のページ分割されたリストを返します。

![List](images/mcp-server-list-tools.png)

### MCP ターゲットの InvokeTool（tools/call）動作

MCP ターゲットの InvokeTool 操作は、ListTools を通じて発見されたツールの実際の実行を処理し、ターゲット MCP サーバーとのリアルタイム通信を管理します。キャッシュベースの ListTools 操作とは異なり、tools/call は MCP サーバーとのアクティブな通信を必要とし、特定の認証、セッション管理、エラー処理の要件が発生します。tools/call リクエストが到着すると、Gateway はまず同期された定義にツールが存在することを検証します。MCP ターゲットの場合、Gateway は MCP サーバーとのセッションを確立するために初期の 'initialize' 呼び出しを実行します。ターゲットが OAuth 認証情報で設定されている場合、Gateway は initialize 呼び出しを行う前に AgentCore Identity から新しい認証情報を取得します。これにより、ListTools が期限切れの認証情報を持つキャッシュされたツールを返した場合でも、実際の呼び出しは有効な認証を使用することが保証されます。

![Invoke](images/mcp-server-invoke-tool.png)


### チュートリアル詳細


| 情報                 | 詳細                                                       |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ  | インタラクティブ                                           |
| AgentCore コンポーネント | AgentCore Gateway、AgentCore Identity                   |
| エージェントフレームワーク | Strands Agents                                        |
| Gateway ターゲットタイプ | MCP サーバー                                             |
| エージェント         | Strands                                                   |
| インバウンド認証 IdP  | Amazon Cognito（他も使用可能）                             |
| アウトバウンド認証    | Amazon Cognito（他も使用可能）                             |
| LLM モデル           | Anthropic Claude Sonnet 4                                 |
| チュートリアル構成    | AgentCore Gateway の作成と呼び出し                         |
| チュートリアル分野    | クロスバーティカル                                         |
| 例の複雑さ           | 簡単                                                       |
| 使用 SDK             | boto3                                                     |

## チュートリアルアーキテクチャ

このチュートリアルは、より広範なエンタープライズの課題の実践的な例として機能します：**既存の MCP サーバーを中央集権型の Gateway アーキテクチャに統合する方法。**

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Jupyter notebook（Python カーネル）
* uv
* AWS 認証情報
* Amazon Cognito

In [ ]:
# Install from the requirements file or pyproject.toml file in current directory
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
import os
# Set AWS credentials if not using SageMaker notebook
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
# Import utils
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

# Setup logging 
import logging

# Configure logging for notebook environment
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

# Set specific logger levels
logging.getLogger("strands").setLevel(logging.INFO)


## Gateway が引き受ける IAM ロールの作成

In [ ]:
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-mcpgateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

## Gateway へのインバウンド認可用の Amazon Cognito プールの作成

In [ ]:
# Creating Cognito User Pool 
import os
import boto3

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "invoke",  # Just 'invoke', will be formatted as resource_server_id/invoke
    "ScopeDescription": "Scope for invoking the agentcore gateway"},
]

scope_names = [f"{RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in SCOPES]
scopeString = " ".join(scope_names)


cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
gw_user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {gw_user_pool_id}")

utils.get_or_create_resource_server(cognito, gw_user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

gw_client_id, gw_client_secret = utils.get_or_create_m2m_client(cognito, gw_user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID, scope_names)

print(f"Client ID: {gw_client_id}")

# Get discovery URL
gw_cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{gw_user_pool_id}/.well-known/openid-configuration'
print(gw_cognito_discovery_url)

## Runtime へのインバウンド認可用の Amazon Cognito プールの作成
このセクションでは、Gateway がターゲットシステムと異なるインバウンド認証を持つ機能を強調しています。これにより、エージェントは単一のインターフェースを通じて複数のアイデンティティプロバイダーを使用するツールにアクセスできます。

In [ ]:
# Creating Cognito User Pool
import os
import boto3

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-runtime-pool"
RESOURCE_SERVER_ID = "sample-agentcore-runtime-id"
RESOURCE_SERVER_NAME = "sample-agentcore-runtime-name"
CLIENT_NAME = "sample-agentcore-runtime-client"
SCOPES = [
    {"ScopeName": "invoke",  # Just 'invoke', will be formatted as resource_server_id/invoke
    "ScopeDescription": "Scope for invoking the agentcore gateway"},
]

scope_names = [f"{RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in SCOPES]
runtimeScopeString = " ".join(scope_names)


cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
runtime_user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {runtime_user_pool_id}")

utils.get_or_create_resource_server(cognito, runtime_user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

runtime_client_id, runtime_client_secret = utils.get_or_create_m2m_client(cognito, runtime_user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID, scope_names)

print(f"Client ID: {runtime_client_id}")

# Get discovery URL
runtime_cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{runtime_user_pool_id}/.well-known/openid-configuration'
print(runtime_cognito_discovery_url)

## Gateway の作成

In [ ]:
# CreateGateway with Cognito authorizer. Use the Cognito user pool created in the previous step
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [gw_client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": gw_cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='ac-gateway-mcp-server',
    roleArn=agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway
    protocolType='MCP',
    protocolConfiguration={
        'mcp': {
            'supportedVersions': ['2025-03-26'],
            'searchType': 'SEMANTIC'
        }
    },
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config,
    description='AgentCore Gateway with MCP Server target'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

## サンプル MCP サーバーを作成して Runtime でホストする

以下を既に存在する MCP サーバーに置き換えることができます。このセクションは [Amazon Bedrock AgentCore Runtime での MCP サーバーのホスティング](https://github.com/awslabs/amazon-bedrock-agentcore-samples/blob/main/01-tutorials/01-AgentCore-runtime/02-hosting-MCP-server/hosting_mcp_server.ipynb) チュートリアルの例に従っています。

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def getOrder() -> int:
    """Get an order"""
    return 123

@mcp.tool()
def updateOrder(orderId: int) -> int:
    """Update existing order"""
    return 456

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

次に、AgentCore Runtime 環境を設定します。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")
agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            runtime_client_id
        ],
        "discoveryUrl": runtime_cognito_discovery_url
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="ac_gateway_mcp_server"
)
print("Configuration completed ✓")

次に、AgentCore Runtime にデプロイします。

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()

agent_arn = launch_result.agent_arn
agent_id = launch_result.agent_id

encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')

agent_url = f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT'
print("Launch completed ✓")
print(f"Agent ARN: {agent_arn}")
print(f"Agent ID: {agent_id}")

## AgentCore Gateway のターゲットとして MCP サーバーを作成する

### アウトバウンド認証の設定

まず、AgentCore Gateway が AgentCore Runtime 内の MCP サーバーへのアウトバウンド認証として使用する AgentCore Identity リソース認証情報プロバイダーを作成します。

In [ ]:
import boto3
identity_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

cognito_provider = identity_client.create_oauth2_credential_provider(
    name="ac-gateway-mcp-server-identity",
    credentialProviderVendor="CustomOauth2",
    oauth2ProviderConfigInput={
        'customOauth2ProviderConfig': {
            'oauthDiscovery': {
                'discoveryUrl': runtime_cognito_discovery_url,
            },
            'clientId': runtime_client_id,
            'clientSecret': runtime_client_secret
        }
    }
)
cognito_provider_arn = cognito_provider['credentialProviderArn']
print(cognito_provider_arn)

### Gateway ターゲットの作成

In [ ]:
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
create_gateway_target_response = gateway_client.create_gateway_target(
    name='mcp-server-target',
    gatewayIdentifier=gatewayID,
    targetConfiguration={
        'mcp': {
            'mcpServer': {
                'endpoint': agent_url
            }
        }
    },
    credentialProviderConfigurations=[
        {
            'credentialProviderType': 'OAUTH',
            'credentialProvider': {
                'oauthCredentialProvider': {
                    'providerArn': cognito_provider_arn,
                    'scopes': [
                        runtimeScopeString
                    ]
                }
            }
        },
    ]
)
print(create_gateway_target_response)

#### Gateway ターゲットが存在し、READY であることを確認する

In [ ]:
list_targets_response = gateway_client.list_gateway_targets(gatewayIdentifier=gatewayID)
print(list_targets_response)

#### インバウンド認可用に Amazon Cognito からアクセストークンをリクエストする

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propagation completes")
token_response = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, scopeString, REGION)
token = token_response["access_token"]
print("Token response:", token)

## Gateway を通じて MCP サーバーツールを検索する

In [ ]:
import requests
import json

def search_tools(gateway_url, access_token, query):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    payload = {
        "jsonrpc": "2.0",
        "id": "search-tools-request",
        "method": "tools/call",
        "params": {
            "name": "x_amz_bedrock_agentcore_search",
            "arguments": {
                "query": query
            }
        }
    }

    response = requests.post(gateway_url, headers=headers, json=payload)
    return response.json()

# Example usage
token_response = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, scopeString, REGION)
access_token = token_response['access_token']
results = search_tools(gatewayURL, access_token, "orders")
print(json.dumps(results, indent=2))

AgentCore Gateway の検索機能に関するより詳細な例については、[03-search-tools](../03-search-tools) を参照してください。

## エージェントを使用して Gateway 経由で注文を操作する

In [ ]:
import json

import requests
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent


def get_token():
    token = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, scopeString, REGION)
    return token['access_token']


def create_streamable_http_transport():
    return streamablehttp_client(
        gatewayURL, headers={"Authorization": f"Bearer {get_token()}"}
    )


client = MCPClient(create_streamable_http_transport)

## The IAM group/user/ configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0", # may need to update model_id depending on region
    temperature=0.7,
    max_tokens=500,  # Limit response length
)

with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(
        model=yourmodel, tools=tools
    )  ## you can replace with any model you like
    # Invoke the agent with the sample prompt. This will only invoke MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi, can you list all tools available to you")
    # Simplified prompt and error handling
    result = agent("Update order 123")

## クリーンアップ
実行した例に応じて、IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、S3 バケットなどの追加リソースも作成されます。これらはクリーンアップの一部として手動で削除する必要がある場合があります。

### 注意：次のノートブック [02-mcp-target-synchronization](02-mcp-target-synchronization.ipynb) に進む場合は、次のチュートリアル用にこれらのリソースを保持することをお勧めします。

### Gateway の削除（オプション）

In [ ]:
utils.delete_gateway(gateway_client, gatewayID)

### アイデンティティプロバイダーの削除（オプション）

In [ ]:
identity_client.delete_oauth2_credential_provider(name='ac-gateway-mcp-server-identity')

### Runtime の削除（オプション）

In [ ]:
import boto3
runtime_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
runtime_client.delete_agent_runtime(agentRuntimeId=agent_id)